<a href="https://colab.research.google.com/github/olivemideva/python_scripts/blob/main/FirstNLPLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NLP
* NLTK = !pip install nltk -q
* Datasets = !pip install datasets -q
* SPACY
* Tesnsorflow
* Pytorch
* Transformers (HF)

In [ ]:
!pip install nltk -q
!pip install datasets -q

     |████████████████████████████████| 441 kB 15.5 MB/s 
     |████████████████████████████████| 212 kB 50.9 MB/s 
     |████████████████████████████████| 115 kB 41.3 MB/s 
     |████████████████████████████████| 95 kB 5.2 MB/s 
     |████████████████████████████████| 163 kB 62.4 MB/s 
     |████████████████████████████████| 127 kB 36.6 MB/s 
     |████████████████████████████████| 115 kB 63.4 MB/s 


In [ ]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#download dataset from huggingface
from datasets import load_dataset

dataset = load_dataset("tweet_eval", "emoji")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
#see contents of downloaded dataset
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})

In [ ]:
tr_text = dataset["train"]["text"]
tr_label = dataset["train"]["label"]

ts_text = dataset["test"]["text"]
ts_label = dataset["test"]["label"]

v_text = dataset["validation"]["text"]
v_label = dataset["validation"]["label"]

In [ ]:
import pandas as pd

df = pd.DataFrame()

#add rows and columns
df['text'] = tr_text + ts_text + v_text
df['label'] = tr_label + ts_label + v_label
df.head(10)

,text,label
0,Sunday afternoon walking through Venice in the...,12
1,Time for some BBQ and whiskey libations. Chomp...,19
2,Love love love all these people ️ ️ ️ #friends...,0
3,"️ ️ ️ ️ @ Toys""R""Us",0
4,Man these are the funniest kids ever!! That fa...,2
5,"#sandiego @ San Diego, California",11
6,My little ️ ️ ️ ️ ️ #ObsessedWithMyDog @ Cafe ...,0
7,"More #tinyepic things #tinyepicwestern, this o...",19
8,Last night ️ @ Omnia Night Club At Caesars Palace,0
9,friendship at its finest. ....#pixar #toystory...,7


In [ ]:
df.shape

(100000, 2)

In [ ]:
#check distribution of label
df['label'].value_counts()

0     21058
1     10252
2      9751
4      6105
7      5293
3      4956
6      4278
18     4056
5      3983
10     3573
17     3063
11     3038
8      3013
9      2737
14     2659
16     2640
12     2640
19     2332
15     2326
13     2247
Name: label, dtype: int64

We pick label 14 and 17

In [ ]:
#create new dataframe
df2 = df[df["label"].isin([14, 17])]
df2.head()

,text,label
31,#Lunch @user best #Ribs in #SanDiego (at @user...,14
35,Surprised everyone by actually surviving a wee...,14
40,Stay classy #SanDiego (@ San Diego Internation...,14
74,@user hi from Tijuana B.C. See you soon,14
77,Birthday cheers with my favorite people!!#26ag...,14


In [ ]:
df2['label'].value_counts()

17    3063
14    2659
Name: label, dtype: int64

In [ ]:
#pre-processing
sample = df2['text'].iloc[0]
print(sample)

#Lunch @user best #Ribs in #SanDiego (at @user in San Diego, CA)


In [ ]:
#cleaning
#remove @
print(str(sample).replace("@", ""))

#Lunch user best #Ribs in #SanDiego (at user in San Diego, CA)


In [ ]:
#remove #
#this overwrites the previous removed @ hence look at the next cell
print(str(sample).replace("#", ""))

Lunch @user best Ribs in SanDiego (at @user in San Diego, CA)


In [ ]:
#cleaning
df2['text'] = df2['text'].str.replace("@", "")
df2['text'] = df2['text'].str.replace("#", "")
df2['text'] = df2['text'].str.replace("\d+", "")#gets rid of numbers
df2['text'] = df2['text'].str.replace("http\S+|www.\S+", "")
df2['text'] = df2['text'].str.lower()#convert to lowercase
df2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/usr/local/li

,text,label
31,lunch user best ribs in sandiego (at user in s...,14
35,surprised everyone by actually surviving a wee...,14
40,stay classy sandiego ( san diego international...,14
74,user hi from tijuana b.c. see you soon,14
77,birthday cheers with my favorite people!!again...,14


In [ ]:
df2.shape

(5722, 2)

In [ ]:
#BoW
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()
bow.fit(df2['text'])
bow = bow.transform(df2['text'])

print(bow.shape)

(5722, 11801)


In [ ]:
#modelling BoW
from sklearn.model_selection import train_test_split as ts
x_train, x_test, y_train, y_test = ts(bow, df2['label'], test_size = 0.2, random_state = 42)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
from sklearn.metrics import accuracy_score as acc
print (acc(y_test, y_pred))

0.6724890829694323


In [ ]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

#Tf Idf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(
    stop_words=stop,
    ngram_range= (1,1)
)
tfidf.fit(df2['text'])
tfidf = tfidf.transform(df2['text'])
print(tfidf.shape)

(5722, 11668)


In [ ]:
#modelling TF Idf
from sklearn.model_selection import train_test_split as ts
x_train, x_test, y_train, y_test = ts(tfidf, df2['label'], test_size = 0.2, random_state = 42)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
from sklearn.metrics import accuracy_score as acc
print (acc(y_test, y_pred))

0.4707423580786026


In [ ]:
#modelling TF Idf NB
from sklearn.model_selection import train_test_split as ts
x_train, x_test, y_train, y_test = ts(tfidf, df2['label'], test_size = 0.2, random_state = 42)

from sklearn.naive_bayes import MultinomialNB
NB = MultinomialNB()
NB.fit(x_train, y_train)
y_pred = NB.predict(x_test)
from sklearn.metrics import accuracy_score as acc
print (acc(y_test, y_pred))

0.8078602620087336
